# Dependências


In [1]:
!pip -q install langchain openai tiktoken "pinecone-client[grpc]" apache_beam mwparserfromhell cohere python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py

# Carregar *API keys*

In [2]:
from google.colab import files
from dotenv import load_dotenv
files_uploaded = files.upload()
load_dotenv()

Saving .env to .env


True

# Imports

In [3]:
# Gerais
import os
import datetime
from uuid import uuid4
import tiktoken
import pinecone

# LangChain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.schema import BaseOutputParser

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Definição de funções

In [4]:
# Função de comprimento (len) baseado em um tokenizer
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

In [5]:
# Função de inserção de dados em um index do Pinecone
def insert_index(index, texts, metadatas, embeddings_model):
    # Obtem ids unicos
    ids = [str(uuid4()) for _ in range(len(texts))]
    # Realiza embedding dos textos
    embeds = embeddings_model.embed_documents(texts)
    # Insere no índice
    index.upsert(vectors=zip(ids, embeds, metadatas))

In [6]:
# Função que itera sobre os chunks para inseri-los no Pinecone
def insert_pinecone(index, chunks, embedding_model):
  batch_limit = 100
  texts = []
  metadatas = []

  # Chunks obtidos na módulo de divisao
  for chunk in chunks:
      # Obter texto e metadados do chunk
      chunk_text = chunk.page_content
      chunk_metadatas = chunk.metadata
      chunk_metadatas["year"] = datetime.datetime.now().year
      chunk_metadatas["text"] = chunk_text

      # Adiciona a lista de textos e metadados
      texts.append(chunk_text)
      metadatas.append(chunk_metadatas)

      # Se ultrapassou o tamanho limite do lote (batch),
      # insere no índice
      if len(texts) >= batch_limit:
          insert_index(index, texts, metadatas, embedding_model)
          # Esvazia listas
          texts = []
          metadatas = []

  # Se ainda restam dados a serem inseridos
  if len(texts) > 0:
      insert_index(index, texts, metadatas, embedding_model)

# Alimentar banco de dados


In [7]:
# Carregar documentos

loader = WebBaseLoader([
    "https://jornal.usp.br/ciencias/ciencias-isolamento-e-coesao-dos-grupos-de-direita-facilitaram-propagacao-coordenada-nas-eleicoes/",
    "http://www.saocarlos.usp.br/atencao-a-saude-mental-e-inclusao-na-universidade/",
    "https://cemeai.icmc.usp.br/projeto-tematico-une-ciencia-de-dados-e-sociologia-no-mapeamento-da-criminalidade/",
    "https://cemeai.icmc.usp.br/o-avanco-das-pesquisas-matematicas-com-foco-no-espectro-autista/",
    "https://www.icmc.usp.br/noticias/6106-como-construir-um-sistema-computacional-se-uma-falha-pode-ser-fatal"
])

documents = loader.load()

# Definição do tokenizer
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Definição do text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

# Divide o documento em chunks
chunks = text_splitter.split_documents(documents)

# Conecta ao Pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Verifica se é preciso criar o índice
index_name = "noticias-icmc"
if index_name not in pinecone.list_indexes():
    # Cria novo índice
    pinecone.create_index(
        name=index_name,
        metric="cosine", # Metrica de busca
        dimension=1536  # Dimensão do embedding. 1536 para text-embedding-ada-002
    )

# Carrega o índice
index = pinecone.GRPCIndex(index_name)

# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Insere chunks no Pinecone
insert_pinecone(index, chunks, embedding_model)

# Construção de *chains*

### Chain1: k = 3 e exigindo resposta concisa

In [42]:
# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Conecta ao Pinecone
index_name = "noticias-icmc"
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Carrega o index do Pinecone (vector database)
vectorstore = Pinecone.from_existing_index(index_name, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Carrega LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Define template de prompt
template = """Use os seguintes trechos de contexto para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Use no máximo três frases e mantenha a resposta o mais concisa possível.
Contexto: {contexto}
Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)

# Define chain
chain1 = {
    "contexto": itemgetter("pergunta") | retriever,
    "pergunta": itemgetter("pergunta")
} | prompt_template | llm | StrOutputParser()

### Chain2: k = 5 e exigindo resposta concisa

In [43]:
# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Conecta ao Pinecone
index_name = "noticias-icmc"
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Carrega o index do Pinecone (vector database)
vectorstore = Pinecone.from_existing_index(index_name, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Carrega LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Define template de prompt
template = """Use os seguintes trechos de contexto para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Use no máximo três frases e mantenha a resposta o mais concisa possível.
Contexto: {contexto}
Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)

# Define chain
chain2 = {
    "contexto": itemgetter("pergunta") | retriever,
    "pergunta": itemgetter("pergunta")
} | prompt_template | llm | StrOutputParser()

### Chain3: k = 3 e não exigindo resposta concisa

In [44]:
# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Conecta ao Pinecone
index_name = "noticias-icmc"
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Carrega o index do Pinecone (vector database)
vectorstore = Pinecone.from_existing_index(index_name, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Carrega LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Define template de prompt
template = """Use os seguintes trechos de contexto para responder à pergunta no final.
Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Contexto: {contexto}
Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)

# Define chain
chain3 = {
    "contexto": itemgetter("pergunta") | retriever,
    "pergunta": itemgetter("pergunta")
} | prompt_template | llm | StrOutputParser()

# Experiências novas

In [20]:
# Pergunta que não está nos documentos, mas sem especificar para o LLM que ele deve dizer que não sabe
# i.e.: forçar alucinação / resposta errada

In [21]:
# Pergunta direto ao LLM, sem passar o contexto

# Experiências

### Chain1: k = 3 e exigindo resposta concisa

In [ ]:
chain1.invoke({"pergunta": "Que tipo de trabalho conjunto o NEV e o CeMEAI estão desenvolvendo considerando as frentes de pesquisa desse trabalho? Inclua na resposta o que significam as siglas NEV e CeMEAI."})

'O NEV (Núcleo de Estudos da Violência) e o CeMEAI (Centro de Ciências Matemáticas Aplicadas à Indústria) estão desenvolvendo um trabalho conjunto que une ciência de dados e sociologia no mapeamento da criminalidade. Esse trabalho envolve o desenvolvimento de ferramentas específicas para o problema em questão, compartilhamento de bancos de dados e perspectivas analíticas, além de introduzir novas perspectivas de análise para problemas complexos.'

In [34]:
chain1.invoke({"pergunta": "Que tipo de trabalho conjunto o NEV e o CeMEAI estão desenvolvendo considerando as frentes de pesquisa desse trabalho?"})

'O NEV e o CeMEAI estão desenvolvendo um trabalho conjunto que envolve a criação de ferramentas matemáticas e computacionais para abordar questões relacionadas à criminalidade, impunidade e legitimidade das instituições de segurança pública. Esse trabalho também inclui o desenvolvimento de um portal de dados para analisar informações relacionadas à criminalidade e o treinamento multidisciplinar de estudantes e pesquisadores.'

In [ ]:
chain1.invoke({"pergunta": "Que tipo de trabalho conjunto o NEV e o CeMEAI estão desenvolvendo?"})

'O NEV e o CeMEAI estão desenvolvendo um projeto temático de ciência de dados e sociologia, com o objetivo de mapear a criminalidade e introduzir novas perspectivas de análise para problemas complexos.'

In [32]:
chain1.invoke({"pergunta": "O que fez com que os grupos de esquerda tivessem menos sucesso do que os grupos de direitas nas redes sociais durantes as eleições brasileiras de 2022?"})

'Os grupos de esquerda tinham uma estrutura hierárquica menos clara e estavam mais misturados com outras comunidades online, enquanto os grupos de direita eram mais isolados e coesos internamente. Isso permitiu uma hierarquia mais rígida nos grupos de direita, com poucas pessoas comunicando para um grande número de seguidores, facilitando a propagação coordenada de informações.'

In [30]:
chain1.invoke({"pergunta": "Qual a principal característica de um grupo político polarizado?"})

'A principal característica de um grupo político polarizado é o isolamento de outros grupos.'

In [18]:
chain1.invoke({"pergunta": "Onde Bruno nasceu?"})

'Não é possível determinar onde Bruno nasceu com base nos trechos de contexto fornecidos.'

In [14]:
chain1.invoke({"pergunta": "O que ocorrerá nos dias 3 e 4 de outubro de 2023?"})

'Não há informações sobre o que ocorrerá nos dias 3 e 4 de outubro de 2023 nos trechos de contexto fornecidos.'

In [19]:
# Consegue responder mesmo com k = 3
chain1.invoke({"pergunta": "Quando ocorrerá o evento 'Atenção à Saúde Mental e Inclusão na Universidade'?"})

"O evento 'Atenção à Saúde Mental e Inclusão na Universidade' ocorrerá nos dias 3 e 4 de outubro de 2023."

### Chain2: k = 5 e exigindo resposta concisa


In [16]:
# Tive que aumentar de k = 3 para k = 5 para conseguir fazer a chain responder corretamente
# Talvez porque o dado onde a informação está esteja sujo com caracteres desnecessários (\n, telefone, etc...)
chain2.invoke({"pergunta": "O que ocorrerá nos dias 3 e 4 de outubro de 2023?"})

'Nos dias 3 e 4 de outubro de 2023 ocorrerá o evento "Atenção à Saúde Mental e Inclusão na Universidade" no Auditório Prof. Sérgio Mascarenhas, no Instituto de Física da USP de São Carlos.'

### Chain3: k = 3 e não exigindo resposta concisa

In [41]:
chain3.invoke({"pergunta": "Que tipo de trabalho conjunto o NEV e o CeMEAI estão desenvolvendo?"})

'O NEV e o CeMEAI estão desenvolvendo um trabalho conjunto que envolve a união da ciência de dados e da sociologia no mapeamento da criminalidade.'

In [38]:
chain3.invoke({"pergunta": "Que tipo de trabalho conjunto o NEV e o CeMEAI estão desenvolvendo considerando as frentes de pesquisa desse trabalho? Inclua na resposta o que significam as siglas NEV e CeMEAI."})

'O NEV (Núcleo de Estudos da Violência da USP) e o CeMEAI (Centro de Ciências Matemáticas Aplicadas à Indústria) estão desenvolvendo um trabalho conjunto no projeto temático de mapeamento da criminalidade. Esse trabalho inclui a criação de ferramentas específicas para o problema em questão, a realização de novas investigações teóricas e computacionais e a introdução de novas perspectivas de análise para problemas complexos. As siglas NEV e CeMEAI significam respectivamente Núcleo de Estudos da Violência da USP e Centro de Ciências Matemáticas Aplicadas à Indústria.'